In [1]:
#all da imports
import pandas as pd

In [14]:
# each row is a festival screening of a film (so films shown multiple times have multiple rows)
fest_long = pd.read_csv("data/1_film-dataset_festival-program_long.csv")

# each row is a film, where the data for the first festival it was shown at listed (only one row per film)
fest_wide = pd.read_csv("data/1_film-dataset_festival-program_wide.csv")

# merge fest_wide and fest_long on "unique.id", keeping all rows from fest_wide, adding the new festivals from fest_long
fest_combined = fest_wide.merge(fest_long, on="unique.id", how="left").sort_values(by='unique.id').drop_duplicates()

fest_combined.head(10)


,unique.id,imdb.id,title.mixed,prod.year,length.min,length,prod.country.1.en,prod.country.2.en,prod.country.3.en,prod.country.4.en,...,competition,genre,fest.first,fest.sect.first,sample.year.first,fest.show.delay.3year.and.more,film.id,sample.year,fest,fest.sect
0,1,tt2917506,a story of children and film,2013.0,101.0,41 min. or longer,United Kingdom,NaN,NaN,NaN,...,0,other documentary,CANNES,Cannes Classics - Documentaries About Cinema,2013,0.0,f2081,2013,CANNES,Cannes Classics - Documentaries About Cinema
1,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,0,other fiction,TIFF,discovery: 2012,2012,0.0,tffnew391,2012,TIFF,discovery: 2012
2,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,0,other fiction,TIFF,discovery: 2012,2012,0.0,f2023,2013,CANNES,Un Certain Regard
3,100,tt0057494,a legend or was it?,1963.0,83.0,41 min. or longer,Japan,NaN,NaN,NaN,...,0,other fiction,BERLINALE,Forum,2013,1.0,f1160,2013,BERLINALE,Forum
4,1000,tt0032445,somewhere in the netherlands,1940.0,86.0,41 min. or longer,Netherlands,NaN,NaN,NaN,...,0,other fiction,BERLINALE,Retrospective,2013,1.0,f1285,2013,BERLINALE,Retrospective
5,1001,tt3613002,song,2013.0,18.0,40 min. or shorter,United States,NaN,NaN,NaN,...,0,experimental fiction,TIFF,Wavelength,2013,0.0,f3255,2013,TIFF,Wavelength
6,1002,tt3003858,song from the forest,2013.0,97.0,41 min. or longer,Germany,NaN,NaN,NaN,...,1,other documentary,IDFA,IDFA Competition for Feature-Length Documentary,2013,0.0,f6232,2013,IDFA,IDFA Competition for Feature-Length Documentary
7,1003,tt2408026,soongava—dance of the orchids,2012.0,85.0,41 min. or longer,Nepal,NaN,NaN,NaN,...,0,other fiction,FRAMELINE,"World Cinema, Queer Asian Cinema",2013,0.0,f5201,2013,FRAMELINE,"World Cinema, Queer Asian Cinema"
8,1004,tt3091254,soul,2013.0,112.0,41 min. or longer,Taiwan,NaN,NaN,NaN,...,0,other fiction,TIFF,Vanguard,2013,0.0,f3349,2013,TIFF,Vanguard
9,1005,tt2919050,soul food stories,2013.0,60.0,41 min. or longer,Bulgaria,Finland,NaN,NaN,...,1,other documentary,IDFA,IDFA Competition for Mid-Length Documentary,2013,0.0,f6233,2013,IDFA,IDFA Competition for Mid-Length Documentary


In [16]:
#  each row is an award of a given film, including nominations and wins (so films with multiple awards have multiple rows)
imdb_awards = pd.read_csv("data/3_imdb-dataset_awards_long.csv").drop_duplicates()

imdb_info = pd.read_csv("data/3_imdb-dataset_general-info_wide.csv").drop_duplicates()

imdb_combined = imdb_awards.merge(imdb_info, on="imdb.id", how="left")

imdb_combined.head(10)

,imdb.id,award.institution,imdb.event.id,award.imdb.year,awards.nom.win,award,award.category,award.person,imdb.name.id,languages,genres,rating,n.ratings,budget,openingUSA,grossUSA,grossWorld
0,tt0019760,"Cinema Eye Honors Awards, US",ev0001823,2014,Winner,Cinema Eye Honors Award,The Influentials,Dziga Vertov,nm0895048,NaN,Documentary | Music,8.4,22566.0,NaN,NaN,NaN,"$3,959"
1,tt0019760,Village Voice Film Poll,ev0002738,1999,Nominee,VVFP Award,Best Film of the Century,NaN,NaN,NaN,Documentary | Music,8.4,22566.0,NaN,NaN,NaN,"$3,959"
2,tt0024844,Village Voice Film Poll,ev0002738,1999,Nominee,VVFP Award,Best Film of the Century,NaN,NaN,French | Russian,Comedy | Drama | Romance,7.8,14485.0,NaN,NaN,NaN,"$9,505"
3,tt0025456,"National Board of Review, USA",ev0000464,1934,Winner,NBR Award,Best Foreign Film,NaN,NaN,English,Documentary,7.3,1964.0,NaN,NaN,NaN,NaN
4,tt0025456,"National Board of Review, USA",ev0000464,1934,Winner,NBR Award,Top Foreign Films,NaN,NaN,English,Documentary,7.3,1964.0,NaN,NaN,NaN,NaN
5,tt0025456,Venice Film Festival,ev0000681,1934,Winner,Mussolini Cup,Best Foreign Film,Robert J. Flaherty,nm0280904,English,Documentary,7.3,1964.0,NaN,NaN,NaN,NaN
6,tt0032553,"Academy Awards, USA",ev0000003,1941,Nominee,Oscar,Best Picture,NaN,NaN,English | Esperanto,Comedy | Drama | War,8.4,203026.0,"$2,000,000 (estimated)",NaN,NaN,"$969,879"
7,tt0032553,"Academy Awards, USA",ev0000003,1941,Nominee,Oscar,Best Actor in a Leading Role,Charles Chaplin,nm0000122,English | Esperanto,Comedy | Drama | War,8.4,203026.0,"$2,000,000 (estimated)",NaN,NaN,"$969,879"
8,tt0032553,"Academy Awards, USA",ev0000003,1941,Nominee,Oscar,Best Actor in a Supporting Role,Jack Oakie,nm0642988,English | Esperanto,Comedy | Drama | War,8.4,203026.0,"$2,000,000 (estimated)",NaN,NaN,"$969,879"
9,tt0032553,"Academy Awards, USA",ev0000003,1941,Nominee,Oscar,"Best Writing, Original Screenplay",Charles Chaplin,nm0000122,English | Esperanto,Comedy | Drama | War,8.4,203026.0,"$2,000,000 (estimated)",NaN,NaN,"$969,879"


In [32]:
# total rows in both dataframes
print(f"Total rows in imdb_combined: {len(imdb_combined)}")
print(f"Total rows in fest_combined: {len(fest_combined)}")
print("\n")

# percent and count of missing values in both imdb_combined and fest_combined for each column
for df, name in [(imdb_combined, "imdb_combined"), (fest_combined, "fest_combined")]:
    print(f"Missing values in {name}:")
    missing_counts = df.isnull().sum()
    missing_percents = (missing_counts / len(df)) * 100
    missing_df = pd.DataFrame({'missing_count': missing_counts, 'missing_percent': missing_percents})
    print(missing_df[missing_df['missing_count'] > 0])
    print("\n")
    
    

Total rows in imdb_combined: 105599
Total rows in fest_combined: 9969


Missing values in imdb_combined:
                missing_count  missing_percent
award                       3         0.002841
award.category          13987        13.245391
award.person            10078         9.543651
imdb.name.id            10078         9.543651
languages               13683        12.957509
genres                  12154        11.509579
rating                  12875        12.192350
n.ratings               12875        12.192350
budget                  60651        57.435203
openingUSA              51519        48.787394
grossUSA                49621        46.990028
grossWorld              35762        33.865851


Missing values in fest_combined:
                                missing_count  missing_percent
imdb.id                                  1497        15.016551
prod.year                                 179         1.795566
length.min                                306         3.0695

In [44]:
#drop columns with more than 60% missing values in both dataframes
threshold = 60.0
for df in [imdb_combined, fest_combined]:
    missing_percents = (df.isnull().sum() / len(df)) * 100
    cols_to_drop = missing_percents[missing_percents > threshold].index
    df.drop(columns=cols_to_drop, inplace=True)
    
print("Columns after dropping those with >60% missing values:")
print(f"fest_combined columns: {fest_combined.columns.tolist()}")
print(f"imdb_combined columns: {imdb_combined.columns.tolist()}")

fest_combined.head(7)

Columns after dropping those with >60% missing values:
fest_combined columns: ['unique.id', 'imdb.id', 'title.mixed', 'prod.year', 'length.min', 'length', 'prod.country.1.en', 'regions.mena', 'regions.africa', 'regions.asia', 'regions.na', 'regions.eu', 'regions.la', 'regions.ocean', 'director.1', 'animt', 'doc', 'exp', 'fict', 'lgbtq', 'imdb.fest', 'retro.fest.sect', 'competition', 'genre', 'fest.first', 'fest.sect.first', 'sample.year.first', 'fest.show.delay.3year.and.more', 'film.id', 'sample.year', 'fest', 'fest.sect']
imdb_combined columns: ['imdb.id', 'award.institution', 'imdb.event.id', 'award.imdb.year', 'awards.nom.win', 'award', 'award.category', 'award.person', 'imdb.name.id', 'languages', 'genres', 'rating', 'n.ratings', 'budget', 'openingUSA', 'grossUSA', 'grossWorld']


,unique.id,imdb.id,title.mixed,prod.year,length.min,length,prod.country.1.en,regions.mena,regions.africa,regions.asia,...,competition,genre,fest.first,fest.sect.first,sample.year.first,fest.show.delay.3year.and.more,film.id,sample.year,fest,fest.sect
686,1602,tt0000409,sage-femme de première classe,1902.0,3.0,40 min. or shorter,France,0,0,0,...,0,other fiction,CLERMONT-FERRAND,Collections,2013,1.0,f4232,2013,CLERMONT-FERRAND,Collections
165,1139,tt0002206,the great steel secret,1912.0,18.0,40 min. or shorter,Netherlands,0,0,0,...,0,other fiction,CLERMONT-FERRAND,Festival Partners,2013,1.0,f4309,2013,CLERMONT-FERRAND,Festival Partners
276,1238,tt0003419,the student of prague,1913.0,83.0,41 min. or longer,Germany,0,0,0,...,0,other fiction,BERLINALE,Retrospective,2013,1.0,f1282,2013,BERLINALE,Retrospective
827,1728,tt0010375,der magische gürtel,1917.0,45.0,41 min. or longer,Germany,0,0,0,...,0,other documentary,IDFA,IDFA Focus: WWI: the First War on Screen,2013,1.0,f6156,2013,IDFA,IDFA Focus: WWI: the First War on Screen
1816,940,tt0014429,safety last!,1923.0,73.0,41 min. or longer,United States,0,0,0,...,0,other fiction,CANNES,Cinéma De La Plage,2013,1.0,f2111,2013,CANNES,Cinéma De La Plage
152,1128,tt0017925,the general,1926.0,78.0,41 min. or longer,United States,0,0,0,...,0,other fiction,CANNES,Cinéma De La Plage,2013,1.0,f2114,2013,CANNES,Cinéma De La Plage
5952,idfnew1155,tt0019760,man with a movie camera,1929.0,68.0,41 min. or longer,Russia,0,0,0,...,0,other documentary,IDFA,Top 10,2015,1.0,idfnew1155,2015,IDFA,Top 10


In [48]:
imdb_cleaned = imdb_combined.drop(columns=['imdb.event.id', 'award.person', 'imdb.name.id', 'n.ratings'])
fest_cleaned = fest_combined.drop(columns=['title.mixed', 'length.min', 'director.1', 'film.id', 'sample.year'])
fest_cleaned.head(7)

,unique.id,imdb.id,prod.year,length,prod.country.1.en,regions.mena,regions.africa,regions.asia,regions.na,regions.eu,...,imdb.fest,retro.fest.sect,competition,genre,fest.first,fest.sect.first,sample.year.first,fest.show.delay.3year.and.more,fest,fest.sect
686,1602,tt0000409,1902.0,40 min. or shorter,France,0,0,0,0,1,...,0,0,0,other fiction,CLERMONT-FERRAND,Collections,2013,1.0,CLERMONT-FERRAND,Collections
165,1139,tt0002206,1912.0,40 min. or shorter,Netherlands,0,0,0,0,1,...,0,0,0,other fiction,CLERMONT-FERRAND,Festival Partners,2013,1.0,CLERMONT-FERRAND,Festival Partners
276,1238,tt0003419,1913.0,41 min. or longer,Germany,0,0,0,0,1,...,1,1,0,other fiction,BERLINALE,Retrospective,2013,1.0,BERLINALE,Retrospective
827,1728,tt0010375,1917.0,41 min. or longer,Germany,0,0,0,0,1,...,1,0,0,other documentary,IDFA,IDFA Focus: WWI: the First War on Screen,2013,1.0,IDFA,IDFA Focus: WWI: the First War on Screen
1816,940,tt0014429,1923.0,41 min. or longer,United States,0,0,0,1,0,...,1,0,0,other fiction,CANNES,Cinéma De La Plage,2013,1.0,CANNES,Cinéma De La Plage
152,1128,tt0017925,1926.0,41 min. or longer,United States,0,0,0,1,0,...,1,0,0,other fiction,CANNES,Cinéma De La Plage,2013,1.0,CANNES,Cinéma De La Plage
5952,idfnew1155,tt0019760,1929.0,41 min. or longer,Russia,0,0,0,0,1,...,1,0,0,other documentary,IDFA,Top 10,2015,1.0,IDFA,Top 10


In [53]:
for column in imdb_cleaned.columns:
    print(f"Unique values in column '{column}':")
    print(imdb_cleaned[column].unique())
    print("\n")
    
for column in fest_cleaned.columns:
    print(f"Unique values in column '{column}':")
    print(fest_cleaned[column].unique())
    print("\n")

Unique values in column 'imdb.id':
['tt0014429' 'tt0017925' 'tt0019760' ... 'tt9434718' 'tt9589562'
 'tt9804142']


Unique values in column 'award.institution':
['National Film Preservation Board, USA'
 'Online Film & Television Association' 'Cinema Eye Honors Awards, US' ...
 'Montevideo Film Festival for Children and Young People'
 'Sole Luna Doc Film Festival'
 'Zanate Festival de Cine Documental Mexicano']


Unique values in column 'award.imdb.year':
[1994 1989 2009 2014 1999 1934 1936 1937 1995 1939 1941 1940 1961 1974
 1997 1942 2007 1943 1990 1944 2003 2008 1971 1996 1946 1945 1947 1948
 1949 1991 1992 1950 1951 1998 1955 1953 1954 1958 1956 1959 2002 2015
 1957 1993 1960 2001 2000 1962 2018 1963 1964 2016 1965 1966 2019 1967
 1969 1968 1972 1970 1979 2012 1973 2011 1977 1975 1976 2010 2013 2006
 2005 1981 1978 1982 1980 1984 1983 1985 1988 2004 1986 2017 1987]


Unique values in column 'awards.nom.win':
['Winner' 'Nominee']


Unique values in column 'award':
['National Film Reg